In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy

N = 6
nTherm = 300
nMeas = 1000
lamda = N**2
T = 1
h = 1
J = 0.4
beta = 1/T

# A method to set the plot parameters and make them beautiful ;)
def beauty_plot():
    plt.figure(figsize=(10,5))
    plt.minorticks_on()
    plt.rcParams["mathtext.fontset"]="cm"
    plt.rcParams['errorbar.capsize'] = 3
    plt.rcParams['mathtext.rm'] = 'serif'
    font={'family' : 'serif','size'   : 22}
    plt.rc("font",**font)
    plt.xticks(fontsize=22,fontname='DejaVu Serif')
    plt.yticks(fontsize=22,fontname='DejaVu Serif')
    plt.grid(color='black',linestyle=':')

def generate_spin_config(N):
    #spin_config = []
    #for i in range(N):
    #    spin_config.append(([(-1)**i for i in range(N)]))
    return 2*np.random.randint(2, size=(N,N))-1
    #print(spin_config)
    #return np.array(spin_config)


def neighbors(s,x,y):
    left = s[y][x-1]
    right = s[y][(x+1)%N]
    bottom = s[(y+1)%N][x]
    top = s[y-1][x]
    return left, right, bottom, top


def calc_delta_S(s,J,h,x,y,N):
    return 2 * s[y][x]*(J*(np.sum(neighbors(s,x,y))) + h)

def numerical(J, h, N, T):
    spin_conf = generate_spin_config(N)
    plot_x = []
    plot_y = []
    
    for i in range(nTherm):
        x = np.random.randint(0,N)
        y = np.random.randint(0,N)
        delta_S = calc_delta_S(spin_conf,J,h,x,y,N)
        rand = np.random.uniform(0,1)
        if rand <= np.exp(-beta*delta_S): # also accept
            spin_conf[y][x] *= -1
        plot_x.append(i)
        plot_y.append(np.mean(spin_conf))
    #plt.plot(plot_x,plot_y)
    #plt.show()
    
    m=np.array([])
    m_plot_y=np.array([])
    for i in range(nMeas):
        for j in range(lamda):
            x = np.random.randint(0,N)
            y = np.random.randint(0,N)
            delta_S = calc_delta_S(spin_conf,J,h,x,y,N)
            if np.random.uniform(0,1) <= np.exp(-beta*delta_S): # also accept
                spin_conf[y][x] *= -1
        m=np.append(m,spin_conf.mean())
        m_plot_y=np.append(m_plot_y, i)

    return np.mean(m)



def epsilon(J):
    return -J*np.coth(2*J)*(1+2/np.pi*(2*np.tanh(2*J)**2-1)*scipy.special.ellipk(4*np.sech(2*J)**2*np.tanh(2*J)**2))





# Exercise 02 - Simulation of the 2d-Ising-model
The two-dimensional Ising-model is studied via a Markov-Chain-Monte-Carlo-Simulation (MCMC) and the results will be compared to analytically calculated solutions.
## Theory
Similar to the one dimensional case, the hamiltonian for the two-dimensional spin-lattice $\mathbf{s}$ in an external magnetic field $h$ in a heat bath of temperature $T$ and with side lengths $N$ is given by 
$$\mathcal{H}(\mathbf{s})=-J\sum_{\langle x,y\rangle}s_xs_y-h\sum_{x}^Ns_x, $$ where $x$ now numbers the lattice site. $\langle x,y\rangle$ denotes the sum over the nearest neighbors, thus $J$ is the coupling strength between the spins. Periodic boundary conditions are assumed in each direction. The probability of finding a specific spin configuration $\mathbf{s}$ is given by 

$$\mathbf{P}(\mathbf{s})=\exp{\bigg(-\frac{\mathcal{H}(\mathbf{s})}{k_bT}\bigg)}/\sum_{\mathbf{s'}}\exp{\bigg(-\frac{\mathcal{H}(\mathbf{s'})}{k_bT}\bigg)}:=\frac{1}{Z}\exp{\bigg(-\frac{\mathcal{H}(\mathbf{s'})}{k_bT}\bigg)}$$

with the partition function $Z:=\sum_{\mathbf{s'}}\exp{\left(\frac{-\mathcal{H}(\mathbf{s'})}{k_BT}\right)}$. In the thermodynamic limit ($N\to\infty$) solutions for energy per site $\epsilon$ and magnetization per spin $m$ with $h=0$ are known:

$$\epsilon=-J \text{coth}(2 J)\left(1+\frac{2}{\pi}\left(2 \tanh ^{2}(2 J)-1\right) K\left(4 \text{sech}^{2}(2 J) \tanh ^{2}(2 J)\right)\right)$$


$$|m|=\begin{cases}
    \left(1-\frac{1}{\sinh ^{4}(2.7)}\right)^{1 / 8} & \forall J>J_c \\
    0 & \forall J\leq J_c
    \end{cases}
$$
Where $J_c=\frac{1}{2}\log{(1+\sqrt{2})}$ and $K(m)$ is the incomplete elliptic integral of the first kind. 

*4. What is the significance of the critical coupling $J_c$ ?*

$J_c$ is the coupling strength belonging to the critical temperature $T_c$. Below the critical temperature it is $J>J_c$ therefore a net magnetization can be measured. For $J<J_c$ the coupling is not sufficient to reach a net magnetization, i.e. for temperatures larger than the critical temperature.

## Aim of the code

### Numerical approach



In [ ]:
magnetization = []

m = []
h_list = [i for i in np.linspace(-1,1,21)]
N_list = [5, 10, 15, 20]

beauty_plot()
for N in N_list:
    for h in h_list:
        m.append(numerical(J, h, N, T))
    plt.plot(h_list, m,"x", label="$N=%d$"%(N))
    m = []

plt.xlabel("$h$")
plt.ylabel("$\\langle m \\rangle$")
plt.legend()
plt.show()


In [ ]:
np.sum(generate_spin_config(20))

In [ ]:
plt.plot(1,1)

plt.xlabel("$h$")
plt.ylabel("$\\langle m \\rangle$")
plt.legend()
plt.show()